In [ ]:
!pip install mtcnn

In [ ]:
pip install facenet-pytorch

In [ ]:
pip install gradio

In [ ]:
pip install grad-cam

In [ ]:
import os
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
import shutil


class FaceRecognition:
    def __init__(self, output_folder, model_checkpoint, device='cpu'):
        self.output_folder = output_folder
        self.model = InceptionResnetV1(pretrained="vggface2", classify=True, num_classes=1, device=device)
        self.mtcnn = MTCNN(
            post_process=False,
            min_face_size=20,
            thresholds=[0.6, 0.7, 0.7],
            factor=0.709,
            device=device
        )
        self.device = device
        self.load_model_checkpoint(model_checkpoint)

    def load_model_checkpoint(self, model_checkpoint):
        checkpoint = torch.load(model_checkpoint, map_location=torch.device('cpu'))
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(self.device)
        self.model.eval()

    def predict(self, input_image):
        face = self.mtcnn(input_image)
        if face is None:
            raise Exception('No face detected')

        face = face.unsqueeze(0)  # Add the batch dimension
        face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)

        prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()
        prev_face = prev_face.astype('uint8')

        face = face.to(self.device, dtype=torch.float32) / 255.0
        face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy()

        target_layers = [self.model.block8.branch1[-1]]
        use_cuda = True if torch.cuda.is_available() else False
        cam = GradCAM(model=self.model, target_layers=target_layers)
        targets = [ClassifierOutputTarget(0)]

        grayscale_cam = cam(input_tensor=face, targets=targets, eigen_smooth=True)
        grayscale_cam = grayscale_cam[0, :]
        visualization = show_cam_on_image(face_image_to_plot, grayscale_cam, use_rgb=True)
        face_with_mask = cv2.addWeighted(prev_face, 1, visualization, 0.5, 0)

        with torch.no_grad():
            output = torch.sigmoid(self.model(face).squeeze(0))
            prediction = "real" if output.item() < 0.5 else "fake"

            real_prediction = 1 - output.item()
            fake_prediction = output.item()

            confidences = {
                'real': real_prediction,
                'fake': fake_prediction
            }

        return confidences, face_with_mask

    def process_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frame_count = 0
        end_time = cv2.getTickCount() + 30 * cv2.getTickFrequency()  # 30 seconds

        while True:
            if cv2.waitKey(1) == ord('x') or cv2.getTickCount() > end_time:
                break

            ret, frame = cap.read()
            if not ret:
                print("End of video or error reading frame.")
                break

            # Detect faces
            boxes, probs = self.mtcnn.detect(frame)
            if boxes is not None:
                for box in boxes:
                    x, y, width, height = box.astype(int)
                    face = frame[y:y + height, x:x + width]

                    # Save the face
                    cv2.imwrite(os.path.join(self.output_folder, f"face_{frame_count}.jpg"), face)

                    # Draw rectangle around the face
                    cv2.rectangle(frame, (x, y), (x + width, y + height), (255, 0, 0), 2)

            frame_count += 1
            # cv2_imshow(frame)

        # Release video capture
        cap.release()

        print("Faces saved in:", self.output_folder)

    def upload_and_process_video(self):
        # Upload video file
        uploaded = files.upload()

        # Move the uploaded video to the specified folder
        for file_name in uploaded.keys():
            video_path = os.path.join(self.output_folder, file_name)
            with open(video_path, 'wb') as f:
                f.write(uploaded[file_name])
            print("Video uploaded:", file_name)

        # Process the uploaded video
        self.process_video(video_path)

        # Process the extracted faces
        self.process_images()



        # Delete the extracted images and video
        self.delete_images_and_video()

        # Stop the program
        os._exit(0)

    def process_images(self):
        # Get a list of all image files in the output folder
        image_files = [f for f in os.listdir(self.output_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

        # List to store all predictions
        predictions = []

        for image_file in image_files:
            # Open the image
            image_path = os.path.join(self.output_folder, image_file)
            image = Image.open(image_path)

            # Call the predict function
            confidences, image_with_mask = self.predict(image)
            plt.imshow(image_with_mask)
            plt.title(f"Image: {image_file} with Mask Overlay")
            plt.axis('off')
            plt.show()

            # Store the prediction
            if confidences['real'] > confidences['fake']:
                predictions.append("real")
            else:
                predictions.append("fake")

        # Count the votes
        vote_count = {
            "real": predictions.count("real"),
            "fake": predictions.count("fake")
        }

        # Determine the final result
        if vote_count["real"] > vote_count["fake"]:
            final_result = "Real"
        else:
            final_result = "Fake"

        # Print vote count and final result
        print("Vote Count:")
        print("Real:", vote_count["real"])
        print("Fake:", vote_count["fake"])
        print("Final Result:", final_result)

    def delete_images_and_video(self):
        """Delete all image files and video from the output folder."""
        for filename in os.listdir(self.output_folder):
            file_path = os.path.join(self.output_folder, filename)
            if os.path.isfile(file_path):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.avi', '.mp4')):
                    try:
                        os.remove(file_path)
                        print(f"Deleted {file_path}")
                    except Exception as e:
                        print(f"Error deleting {file_path}: {e}")

# Example usage
output_folder = "/content/drive/MyDrive/Colab Notebooks/Images_detection"
model_checkpoint = "/content/drive/MyDrive/Colab Notebooks/resnetinceptionv1_epoch_32.pth"

face_recognition = FaceRecognition(output_folder, model_checkpoint)
face_recognition.upload_and_process_video()
